# Exploration

The data used in this notebook were found online on Free Internet Chess Server (FICS) https://www.ficsgames.org/download.html

Before exploring the data, we need to figure out how the chess library in Python works.

In [70]:
# importing the needed libraries
import pandas as pd 
import chess.pgn
import chess

* Checking how many lines the .pgn file has

In [71]:
! wc -l ~/Downloads/ficsgamesdb_201701_chess_movetimes_1545427.pgn 

 19191183 /Users/danaiavg/Downloads/ficsgamesdb_201701_chess_movetimes_1545427.pgn


* Taking a look at the format of the data, to figure out what we need and what not for our analysis

In [72]:
! head -20 ~/Downloads/ficsgamesdb_201701_chess_movetimes_1545427.pgn 

* Counting how many times the word 'FICSGamesDBGameNo' appears in the file, which indicates 

In [73]:
! grep FICSGamesDBGameNo ~/Downloads/ficsgamesdb_201701_chess_movetimes_1545427.pgn | wc -l

  955855


As we can observe there are 955,855 games for January 2017. 

In [74]:
# reading in the data
pgn = open("/Users/danaiavg/Downloads/ficsgamesdb_201701_chess_movetimes_1545427.pgn")

In order to run our analysis faster we are going to get a subset of data.

Notes:
- When there are a lot of moves, the following doesn't show until the check mate, wierd

Following there is an example of how to print a hierarchical chart 
https://plot.ly/python/tree-plots/

another example in this link: https://plot.ly/~empet/14305/a-tree-with-improved-walker-layout-1/#plot

## Trying to create the list to viz

In [75]:
first_game = chess.pgn.read_game(pgn)
board = chess.Board()
moves = first_game.main_line()
node = first_game
while node.variations:
    next_node = node.variation(0)
#     print(next_node)

    print(next_node.move)  #prints the origin square and the destination square
    b = next_node.move.to_square
#     print(b)
    a = node.board().san(next_node.move)[-2:]
    print(a)
    board.push_san(node.board().san(next_node.move))
    print(board.piece_at(next_node.move.to_square))  #prints the piece that was moved
    
    print(board)
    
    print(board.is_game_over())
    node = next_node

d2d4
d4
P
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
False
d7d5
d5
p
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
False
b1c3
c3
N
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . P . . . .
. . N . . . . .
P P P . P P P P
R . B Q K B N R
False
g8f6
f6
n
r n b q k b . r
p p p . p p p p
. . . . . n . .
. . . p . . . .
. . . P . . . .
. . N . . . . .
P P P . P P P P
R . B Q K B N R
False
c1f4
f4
B
r n b q k b . r
p p p . p p p p
. . . . . n . .
. . . p . . . .
. . . P . B . .
. . N . . . . .
P P P . P P P P
R . . Q K B N R
False
e7e6
e6
p
r n b q k b . r
p p p . . p p p
. . . . p n . .
. . . p . . . .
. . . P . B . .
. . N . . . . .
P P P . P P P P
R . . Q K B N R
False
f2f3
f3
P
r n b q k b . r
p p p . . p p p
. . . . p n . .
. . . p . . . .
. . . P . B . .
. . N . . P . .
P P P . P . P P
R . . Q K B N 

R
. . . . . . . .
. . r . . p k .
. p . . p . p .
p . . p P r . .
P . . P . . . P
. P P . . . . R
. . K . . . . .
. . . . . . . R
False
f5f2
2+
r
. . . . . . . .
. . r . . p k .
. p . . p . p .
p . . p P . . .
P . . P . . . P
. P P . . . . R
. . K . . r . .
. . . . . . . R
False
c2d3
d3
K
. . . . . . . .
. . r . . p k .
. p . . p . p .
p . . p P . . .
P . . P . . . P
. P P K . . . R
. . . . . r . .
. . . . . . . R
False
f2b2
b2
r
. . . . . . . .
. . r . . p k .
. p . . p . p .
p . . p P . . .
P . . P . . . P
. P P K . . . R
. r . . . . . .
. . . . . . . R
False
h4h5
h5
P
. . . . . . . .
. . r . . p k .
. p . . p . p .
p . . p P . . P
P . . P . . . .
. P P K . . . R
. r . . . . . .
. . . . . . . R
False
b2b3
b3
r
. . . . . . . .
. . r . . p k .
. p . . p . p .
p . . p P . . P
P . . P . . . .
. r P K . . . R
. . . . . . . .
. . . . . . . R
False
d3c2
c2
K
. . . . . . . .
. . r . . p k .
. p . . p . p .
p . . p P . . P
P . . P . . . .
. r P . . . . R
. . K . . . . .
. . . . . . . R
False


## Start from here

In [76]:
initial_list = []
for i in range(10):
    first_game = chess.pgn.read_game(pgn)
    board = chess.Board()
    moves = first_game.main_line()
    node = first_game
    a = 0
    list1 = []
    while node.variations:
        next_node = node.variation(0)
        b = str(next_node.move)[-2:]  #take the following position
        list1.append((a,b))
        a = b
        board.push_san(node.board().san(next_node.move))
        node = next_node
        if len(list1) > 3:  #take a subset for depth
            break
    initial_list = initial_list + [i for i in list1 if i not in initial_list]  #there is a concern with that

In [77]:
initial_list

[(0, 'd4'),
 ('d4', 'e5'),
 ('e5', 'c3'),
 ('c3', 'd4'),
 (0, 'e4'),
 ('e4', 'c5'),
 ('c5', 'f3'),
 ('f3', 'c6'),
 ('c5', 'f4'),
 ('f4', 'c6'),
 (0, 'b3'),
 ('b3', 'e6'),
 ('e6', 'b2'),
 ('b2', 'd5'),
 (0, 'f3'),
 ('f3', 'e5'),
 ('e5', 'e4'),
 ('e4', 'f6'),
 ('d4', 'e6'),
 ('e6', 'e4'),
 ('e4', 'h5'),
 ('e6', 'f3'),
 ('f3', 'f5'),
 ('c5', 'b4'),
 ('b4', 'e6'),
 ('d4', 'f6'),
 ('f6', 'e3'),
 ('e3', 'd5'),
 ('d4', 'd6'),
 ('d6', 'f4')]

### Plot - None hierarcical (for now)

In [78]:
import plotly.plotly as py
import plotly.graph_objs as go
import networkx as nx
import matplotlib.pyplot as plt
import collections
from networkx.drawing.nx_agraph import graphviz_layout
init_notebook_mode(connected=True)

In [79]:
node1 = [n[0] for n in initial_list]
node2 = [n[1] for n in initial_list]
node1.extend(node2)
my_nodes = list(set(node1))
my_nodes.sort()

In [80]:
G=nx.Graph()
# my_nodes=range(len(my_nodes))
G.add_nodes_from(my_nodes)
my_edges=initial_list
G.add_edges_from(my_edges)

In [81]:
pos=nx.fruchterman_reingold_layout(G) 

In [82]:
pos=collections.OrderedDict(sorted(pos.items()))

In [83]:
Xn=[pos[k][0] for k in pos.keys()]
Yn=[pos[k][1] for k in pos.keys()]

In [84]:
labels=my_nodes

In [85]:
trace_nodes=dict(type='scatter',
                 x=Xn, 
                 y=Yn,
                 mode='markers',
                 marker=dict(symbol='dot', size=28, color='rgb(0,240,0)'),
                 text=labels,
                 hoverinfo='text')

In [86]:
Xe=[]
Ye=[]
for e in G.edges():
    Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
    Ye.extend([pos[e[0]][1], pos[e[1]][1], None])

In [87]:
trace_edges=dict(type='scatter',
                 mode='lines',
                 x=Xe,
                 y=Ye,
                 line=dict(width=1, color='rgb(25,25,25)'),
                 hoverinfo='none' 
                )

In [88]:
axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )
layout=dict(title= 'Graph',  
            font= dict(family='Balto'),
            width=600,
            height=600,
            autosize=False,
            showlegend=False,
            xaxis=axis,
            yaxis=axis,
            margin=dict(
            l=40,
            r=40,
            b=85,
            t=100,
            pad=0,
       
    ),
    hovermode='closest',
    plot_bgcolor='#efecea', #set background color            
    )


fig = dict(data=[trace_edges, trace_nodes], layout=layout)

In [89]:
def make_annotations(pos, anno_text, font_size=14, font_color='rgb(10,10,10)'):
    L=len(pos)
    if len(anno_text)!=L:
        raise ValueError('The lists pos and text must have the same len')
    annotations = []
    i = 0
    for k in pos.keys():
        annotations.append(dict(text=anno_text[i], 
                                x=pos[k][0], 
                                y=pos[k][1]+0.075,#this additional value is chosen by trial and error
                                xref='x1', yref='y1',
                                font=dict(color= font_color, size=font_size),
                                showarrow=False)
                          )
        i += 1
    return annotations  

In [90]:
fig['layout'].update(annotations=make_annotations(pos, labels))

In [91]:
iplot(fig, filename='Tree-Reingold-Tilf')

Two problems right now:
- recursion
- positioning

For hierarcical networks:
- https://plot.ly/python/tree-plots/ (but I couldn't get igraph to work and recursion is not solved)
- https://stackoverflow.com/questions/29586520/can-one-get-hierarchical-graphs-from-networkx-with-python-3?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa (after that example, I think we really need to create our own function)